# Necessary imports

In [1]:
%pip install pandas
%pip install numpy
%pip install scikit-learn
%pip install optuna
%pip install mlflow
%pip install holidays

import pandas as pd
import numpy as np
from datetime import datetime, time
import holidays
import optuna

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Carregamento de dados

In [2]:
dtypes = {
    "hora_minuto":  str,
    "municipio": "category",
    "bairro": "category",
    "endereco": "category",
    "origem_chamado": "category",
    "tipo": "category",
    "subtipo": "category",
    "sexo": "category",
    "idade": float,
    "motivo_finalizacao": "category",
    "motivo_desfecho": "category",
}

columns_to_datetime = ["data"]

raw_df = pd.read_csv("./datasets/ocorrencias2022.csv", sep=';', dtype=dtypes, parse_dates=columns_to_datetime)
raw_df['hora_minuto'] = pd.to_datetime(raw_df['hora_minuto']).dt.time

# Tratando dados vazios e inconsistentes

In [3]:
raw_df = raw_df.drop("motivo_finalizacao", axis=1)
raw_df = raw_df.dropna(subset=["municipio", "bairro", "subtipo", "sexo", "idade"])

def older_than_120(age: int):
  if age >= 120:
    return 120
  return age

raw_df["idade"] = raw_df["idade"].apply(lambda x : older_than_120(x))

In [4]:
raw_df.head()

,data,hora_minuto,municipio,bairro,endereco,origem_chamado,tipo,subtipo,sexo,idade,motivo_desfecho
0,2022-01-01,00:02:19,RECIFE,JARDIM SAO PAULO,R LEANDRO BARRETO,RESIDENCIAL,RESPIRATORIA,CASO SUSPEITO COVID-19,FEMININO,81.0,PACIENTE RECUSA SER REMOVIDO
1,2022-01-01,00:03:00,RECIFE,MADALENA,R ALTINHO,RESIDENCIAL,CAUSAS EXTERNAS,QUEDA DA PROPRIA ALTURA,FEMININO,81.0,DESISTÊNCIA DA SOLICITAÇÃO
2,2022-01-01,00:09:38,MACAPARANA,CENTRO,R POSSIDONIO JULIAO DA SILVA,VIA PÚBLICA,CAUSAS EXTERNAS,OUTROS,MASCULINO,0.0,SEM DESFECHO
4,2022-01-01,00:12:16,PALMARES,PALMARES CENTRO,RUA DOUTOR COSTA MAIA,RESIDENCIAL,GERAIS/OUTROS,OUTROS,MASCULINO,84.0,OCORRÊNCIA CONCLUÍDA COM ÊXITO
5,2022-01-01,00:13:00,LIMOEIRO,GAMELEIRA (ZONA RURAL),PE,VIA PÚBLICA,CAUSAS EXTERNAS,ACIDENTE DE TRANSITO ENVOLVENDO MOTO,MASCULINO,0.0,OCORRÊNCIA CONCLUÍDA COM ÊXITO


In [5]:
motivos_desfecho = raw_df['motivo_desfecho'].value_counts()
motivos_desfecho

OCORRÊNCIA CONCLUÍDA COM ÊXITO                           35450
SEM DESFECHO                                             33077
REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES            6384
PACIENTE RECUSA SER REMOVIDO                              5089
DESISTÊNCIA DA SOLICITAÇÃO                                4439
NÃO HÁ PACIENTE NO ENDEREÇO                               1781
PACIENTE JÉ ENCONTRADO EM ÓBITO                           1647
CASA FECHADA / NINGUÉM ATENDE AO CHAMADO                  1207
REMOVIDO PELOS BOMBEIROS/CIODS                            1172
ACOMPANHANTE RECUSA REMOÇÃO                                927
PACIENTE NÃO NECESSITA DE REMOÇÃO                          572
SOLICITAÇÃO DUPLICADA                                      406
ÓBITO DURANTE O ATENDIMENTO                                133
TROTE                                                      113
PACIENTE SEM CONDIÇÕES CLÍNICAS DE REMOÇÃO HOSPITALAR       62
Name: motivo_desfecho, dtype: int64

# Propósito do modelo

Quando um chamado é iniciado, a maior partes das informações é recolhida na hora. O motivo do desfecho, contudo, só pode ser preenchido após o encerramento do chamado. Destacamos os seguintes motivos de desfecho:

"PACIENTE JÉ ENCONTRADO EM ÓBITO"

"ÓBITO DURANTE O ATENDIMENTO"

Partimos do pressuposto de que alguns desses casos de óbito poderiam ser evitados com maior agilidade ou priorização por parte do SAMU.

Daí veio a idéia do nosso modelo:

Um modelo capaz de determinar com certo grau de certeza, baseando-se nos detalhes recolhidos na hora do registro da ocorrência, se aquela ocorrência corre risco de terminar com algum óbito. Caso ela afirme positivamente, essa informação poderia ser usada para maior priorização ou agilidade por parte da equipe.

# Criação do dataset de treino

1. Inicialmente, iremos criar uma coluna de "obito" mais simples, que engloba os 2 tipos de motivo de desfecho que levaram a óbitos, é composta por 1 ou 0, em caso de óito ou o contrário.

2. Após isso, removeremos a coluna "motivo_desfecho", pois, como discutido, ela só é preenchida após a conclusão da ocorrência, então não faria sentido o modelo ter acesso a essa informação no momento em que analisa uma ocorrência nova.

3. Criaremos uma coluna que transforme o dado de hora numa relação mais categórica e genérica, "Período", como madrugada, manhã, tarde e noite.

4. Extrairemos dados como dia da semana e se o dia era feriado a partir da data.

5. Removeremos colunas que julgamos serem irrelevantes para a classificação, como data e colunas de endereço. Decidimos manter a coluna de hora_minuto pois é argumentável que a hora que algo ocorre pode impactar na conclusão.

6. Para utilização do scikit learn, utilizaremos a técnica One Hot Enconding para transformar cada tipo categórico em um formato mais desejável para o scikit learn.
É importante ressaltar que os dados categóricos "Período" e "Idade" são ordinais, isso é, uma certa ordem pode ser determinada entre as categorias (manhã vem antes da tarde que vem antes da noite, 55 anos vem antes dos 56 anos), porém o resto das colunas representam dados nominais.

7. Por fim, separamos os datasets para treino, validação e teste. Porém, como existe uma disparidade gigantesca entre os rótulos (existem muito mais dados com o rótulo "negativo" do que com o rótulo "positivo"), fomos aconselhados a balancear os dados de treinamento. Para isso, constatamos a quantidade de rótulos "positivos" presentes no conjunto de treinamento, e geramos uma amostra com esse mesmo número a partir dos rótulos "negativos", juntando tudo no final. O resultado é que teremos um conjunto de treinamento bem menor do que se tivéssemos mantido o conjunto desbalaceado, porém, ter exemplos justos de cada rótulo é potencialmente mais positivo para o modelo do que apenas um conjunto enorme, porém desbalanceado.

In [6]:
raw_df['obito'] = [1 if x == "PACIENTE JÉ ENCONTRADO EM ÓBITO" or x == "ÓBITO DURANTE O ATENDIMENTO" else 0 for x in raw_df['motivo_desfecho']]

madrugada_upper = time(4, 59, 59)
manha_upper = time(11, 59, 59)
tarde_upper = time(17, 59, 59)
noite_upper = time(23, 59, 59)

conditions = [
    (raw_df['hora_minuto'] <= madrugada_upper),
    ((raw_df['hora_minuto'] > madrugada_upper) & (raw_df['hora_minuto'] <= manha_upper)),
    ((raw_df['hora_minuto'] > manha_upper) & (raw_df['hora_minuto'] <= tarde_upper)),
    ((raw_df['hora_minuto'] > tarde_upper) & (raw_df['hora_minuto'] <= noite_upper)),
]
choices = ['madrugada', 'manha', 'tarde', 'noite']
raw_df['periodo'] = np.select(conditions, choices, default='indeterminado')

recife_holidays = holidays.Brazil(years=[2022])

raw_df['dia_semana'] = raw_df['data'].dt.day_of_week
raw_df['feriado'] = [1 if x in recife_holidays else 0 for x in raw_df['data']]

relevant_columns = [
    "dia_semana",
    "feriado",
    "periodo",
    "origem_chamado",
    "tipo",
    "subtipo",
    "sexo",
    "idade",
    "obito"
]
df = raw_df[relevant_columns]
df.head()

,dia_semana,feriado,periodo,origem_chamado,tipo,subtipo,sexo,idade,obito
0,5,1,madrugada,RESIDENCIAL,RESPIRATORIA,CASO SUSPEITO COVID-19,FEMININO,81.0,0
1,5,1,madrugada,RESIDENCIAL,CAUSAS EXTERNAS,QUEDA DA PROPRIA ALTURA,FEMININO,81.0,0
2,5,1,madrugada,VIA PÚBLICA,CAUSAS EXTERNAS,OUTROS,MASCULINO,0.0,0
4,5,1,madrugada,RESIDENCIAL,GERAIS/OUTROS,OUTROS,MASCULINO,84.0,0
5,5,1,madrugada,VIA PÚBLICA,CAUSAS EXTERNAS,ACIDENTE DE TRANSITO ENVOLVENDO MOTO,MASCULINO,0.0,0


In [7]:
ohe_features = [
    "dia_semana",
    "feriado",
    "periodo",
    "origem_chamado",
    "tipo",
    "subtipo",
    "sexo",
]

ohe_df = pd.get_dummies(df, prefix=ohe_features, columns=ohe_features)
ohe_df.head()

,idade,obito,dia_semana_0,dia_semana_1,dia_semana_2,dia_semana_3,dia_semana_4,dia_semana_5,dia_semana_6,feriado_0,...,subtipo_TONTURAS,subtipo_TRABALHO DE PARTO,subtipo_TRAUMA OCULAR,subtipo_TREMORES,subtipo_USO DE DROGAS ILICITAS,subtipo_VOMITOS,subtipo_CORPO ESTRANHO OCULAR,subtipo_DOENCAS SEXUALMENTE TRANSMISSIVEIS,sexo_FEMININO,sexo_MASCULINO
0,81.0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,81.0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0.0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,84.0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0.0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
from sklearn.model_selection import train_test_split

features = list(ohe_df.columns)
features.remove('obito')

X = ohe_df[features].to_numpy()

y = ohe_df['obito'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.125, random_state=1337)

# Balanceando dataset

Será necessário balancear o dataset para termos um número similar de exemplos de cada label que desejamos classificar.

In [9]:
train_df = pd.DataFrame(X_train, columns=features)
train_df = pd.concat([train_df, pd.DataFrame(y_train, columns=['obito'])], axis=1)

negative_df = train_df[train_df['obito'] == 0]
positive_df = train_df[train_df['obito'] == 1]

positive_df_count = len(positive_df.index)
negative_df_sample = negative_df.sample(n=positive_df_count, random_state=1337)

print("rows in the positive label samples:", positive_df_count)
print("rows in the negative label samples:", len(negative_df_sample.index))

balanced_df = pd.concat([positive_df, negative_df_sample])
# This essentially shuffles the df.
balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)
print("Total number of training samples:", len(balanced_df.index))

X_train = balanced_df[features].to_numpy()

y_train = balanced_df['obito'].to_numpy()

rows in the positive label samples: 1249
rows in the negative label samples: 1249
Total number of training samples: 2498


In [10]:
print("X_train count:", X_train.shape)
print("y_train count:", y_train.shape)
print("X_validation count:", X_validation.shape)
print("y_validation count:", y_validation.shape)
print("X_test count:", X_test.shape)
print("y_test count:", y_test.shape)

X_train count: (2498, 147)
y_train count: (2498,)
X_validation count: (9246, 147)
y_validation count: (9246,)
X_test count: (18492, 147)
y_test count: (18492,)


# Seleção e otimização de modelos

Os 4 modelos escolhidos foram:
- Naive Bayes (Gaussian)
- Random Forest
- Decision Tree
- Non-Linear SVC

Além disso, utilizamos o método GridSearch para otimização dos hiper-parâmetros.

In [11]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score
from urllib.parse import urlparse

labels = [0, 1]

# Configurando MLFlow

In [12]:
import mlflow
from mlflow import MlflowClient

def fetch_logged_data(run_id):
    client = MlflowClient()
    data = client.get_run(run_id).data
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in client.list_artifacts(run_id, "model")]
    return data.params, data.metrics, tags, artifacts

mlflow.sklearn.autolog()

training_time_in_seconds = 600

# Using optuna to elect best model and hyper-parameters

Modelo 1: Naive Bayes

In [13]:
model = Pipeline([
    ('clf', GaussianNB())
])

def objective(trial):    
    with mlflow.start_run() as run:

        clf__priors = trial.suggest_categorical('clf__priors', [None])
        clf__var_smoothing = trial.suggest_float('clf__var_smoothing', -9, 0)

        params = {
        'clf__priors': clf__priors,
        'clf__var_smoothing': clf__var_smoothing
        }

        model.set_params(**params)

        cv_score = np.mean(cross_val_score(model, X_train, y_train, cv=8, n_jobs=-1))
        mlflow.log_metric("cross_val_score", cv_score)

        model.fit(X_train, y_train)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(model, "model", registered_model_name="GaussianNBModel")
        else:
            mlflow.sklearn.log_model(model, "model")

        pred = model.predict(X_test)
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, pred, labels=labels)

        precision = precision[1]
        recall = recall[1]
        f1 = f1[1]

        mlflow.log_metric("test_precision", precision)
        mlflow.log_metric("test_recall", recall)
        mlflow.log_metric("test_f1", f1)

    return cv_score

study = optuna.create_study()
study.optimize(objective, timeout=training_time_in_seconds)

[I 2022-09-28 18:31:49,779] A new study created in memory with name: no-name-bc410ba7-054e-43d5-9417-1eed716465ba
/home/alps2/.local/lib/python3.10/site-packages/sklearn/naive_bayes.py:487: RuntimeWarning: invalid value encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/home/alps2/.local/lib/python3.10/site-packages/sklearn/naive_bayes.py:487: RuntimeWarning: invalid value encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/home/alps2/.local/lib/python3.10/site-packages/sklearn/naive_bayes.py:487: RuntimeWarning: invalid value encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/home/alps2/.local/lib/python3.10/site-packages/sklearn/naive_bayes.py:487: RuntimeWarning: invalid value encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/home/alps2/.local/lib/python3.10/site-packages/sklearn/naive_bayes.py:487: RuntimeWarning: invalid value encountered in log
  n_ij = -0.5

In [14]:
print("Stats for the optimized Model")
print("Best Score:", study.best_value)
best_parameters = study.best_params
model_parameters = {}
for param in best_parameters:
    print(param, ":", best_parameters[param])
    model_parameters[param[5:]] = best_parameters[param]

tuned_model = GaussianNB(**model_parameters)
tuned_model.fit(X_train, y_train)

pred = tuned_model.predict(X_test)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, pred, labels=labels)
print("labels:", labels)
print("Precision:", [round(x, 2) for x in precision])
print("recall:", [round(x, 2) for x in recall])
print("f1:", [round(x, 2) for x in f1])

2022/09/28 18:41:52 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '917d38a812214b1588dd38f7f98e01ef', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
/home/alps2/.local/lib/python3.10/site-packages/sklearn/naive_bayes.py:487: RuntimeWarning: invalid value encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/home/alps2/.local/lib/python3.10/site-packages/sklearn/naive_bayes.py:487: RuntimeWarning: invalid value encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/home/alps2/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2022/09/28 18:41:52 WARNING mlflow.sklearn.utils: log_loss

Stats for the optimized Model
Best Score: 0.5
clf__priors : None
clf__var_smoothing : -5.686740623152253
labels: [0, 1]
Precision: [0.98, 0.0]
recall: [1.0, 0.0]
f1: [0.99, 0.0]


/home/alps2/.local/lib/python3.10/site-packages/sklearn/naive_bayes.py:487: RuntimeWarning: invalid value encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/home/alps2/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Modelo 2: Random Forest

In [15]:
model = Pipeline([
    ('clf', RandomForestClassifier())
])

def objective(trial):
    with mlflow.start_run() as run:

        clf__n_estimators = trial.suggest_int('clf__n_estimators', 50, 501, 50)
        clf__max_depth = trial.suggest_int('clf__max_depth', 10, 110, log=True) # Find a way to include [None]. Maybe a categorical?
        clf__max_features = trial.suggest_categorical('clf__max_features', ['sqrt', 'log2', None])

        params = {
            'clf__n_estimators': clf__n_estimators,
            'clf__max_depth': clf__max_depth,
            'clf__max_features': clf__max_features
        }
        
        model.set_params(**params)

        cv_score = np.mean(cross_val_score(model, X_train, y_train, cv=8, n_jobs=-1))
        mlflow.log_metric("cross_val_score", cv_score)

        model.fit(X_train, y_train)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForestModel")
        else:
            mlflow.sklearn.log_model(model, "model")

        pred = model.predict(X_test)
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, pred, labels=labels)

        precision = precision[1]
        recall = recall[1]
        f1 = f1[1]

        mlflow.log_metric("test_precision", precision)
        mlflow.log_metric("test_recall", recall)
        mlflow.log_metric("test_f1", f1)

    return cv_score

study = optuna.create_study()
study.optimize(objective, timeout=(2*training_time_in_seconds))

[I 2022-09-28 18:41:53,528] A new study created in memory with name: no-name-3c292763-64a6-4ed0-8205-c856fd9ee4cd
/home/alps2/.local/lib/python3.10/site-packages/optuna/distributions.py:683: UserWarning: The distribution is specified by [50, 501] and step=50, but the range is not divisible by `step`. It will be replaced by [50, 500].
  warnings.warn(
2022/09/28 18:41:57 WARNING mlflow.sklearn.utils: _get_classifier_artifacts.<locals>.plot_confusion_matrix failed. The artifact training_confusion_matrix will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'
2022/09/28 18:41:57 WARNING mlflow.sklearn.utils: RocCurveDisplay.from_estimator failed. The artifact training_roc_curve will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'
2022/09/28 18:41:58 WARNING mlflow.sklearn.utils: PrecisionRecallDisplay.from_estimator failed. The artifact training_precision_recall_curve will not be recorded. Artifact error: 'NoneType' object has no attr

In [16]:
print("Stats for the optimized Model")
print("Best Score:", study.best_value)
best_parameters = study.best_params
model_parameters = {}
for param in best_parameters:
    print(param, ":", best_parameters[param])
    model_parameters[param[5:]] = best_parameters[param]

tuned_model = RandomForestClassifier(**model_parameters)
tuned_model.fit(X_train, y_train)

pred = tuned_model.predict(X_test)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, pred, labels=labels)
print("labels:", labels)
print("Precision:", [round(x, 2) for x in precision])
print("recall:", [round(x, 2) for x in recall])
print("f1:", [round(x, 2) for x in f1])

2022/09/28 19:02:00 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '62068aa1b79d45aa93154a1d4a8ec283', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Stats for the optimized Model
Best Score: 0.6965777832391251
clf__n_estimators : 250
clf__max_depth : 64
clf__max_features : log2


2022/09/28 19:02:01 WARNING mlflow.sklearn.utils: _get_classifier_artifacts.<locals>.plot_confusion_matrix failed. The artifact training_confusion_matrix will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'
2022/09/28 19:02:01 WARNING mlflow.sklearn.utils: RocCurveDisplay.from_estimator failed. The artifact training_roc_curve will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'
2022/09/28 19:02:01 WARNING mlflow.sklearn.utils: PrecisionRecallDisplay.from_estimator failed. The artifact training_precision_recall_curve will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'


labels: [0, 1]
Precision: [0.99, 0.04]
recall: [0.72, 0.66]
f1: [0.83, 0.08]


Modelo 3: Decision Tree

In [17]:
model = Pipeline([
    ('clf', DecisionTreeClassifier())
])

def objective(trial):
    with mlflow.start_run() as run:

        clf__splitter = trial.suggest_categorical('clf__splitter', ['best', 'random'])
        clf__max_depth = trial.suggest_int('clf__max_depth', 10, 110, log=True) # Find a way to include [None]. Maybe a categorical?
        clf__max_features = trial.suggest_categorical('clf__max_features', ['sqrt', 'log2', None])

        params = {
            'clf__splitter': clf__splitter,
            'clf__max_depth': clf__max_depth,
            'clf__max_features': clf__max_features
        }
        
        model.set_params(**params)

        cv_score = np.mean(cross_val_score(model, X_train, y_train, cv=8, n_jobs=-1))
        mlflow.log_metric("cross_val_score", cv_score)

        model.fit(X_train, y_train)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(model, "model", registered_model_name="DecisionTreeModel")
        else:
            mlflow.sklearn.log_model(model, "model")

        pred = model.predict(X_test)
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, pred, labels=labels)

        precision = precision[1]
        recall = recall[1]
        f1 = f1[1]

        mlflow.log_metric("test_precision", precision)
        mlflow.log_metric("test_recall", recall)
        mlflow.log_metric("test_f1", f1)

    return cv_score

study = optuna.create_study()
study.optimize(objective, timeout=(2*training_time_in_seconds))

[I 2022-09-28 19:02:04,056] A new study created in memory with name: no-name-ca878687-58f3-443f-ac4b-c6fd209e012a
2022/09/28 19:02:04 WARNING mlflow.sklearn.utils: _get_classifier_artifacts.<locals>.plot_confusion_matrix failed. The artifact training_confusion_matrix will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'
2022/09/28 19:02:04 WARNING mlflow.sklearn.utils: RocCurveDisplay.from_estimator failed. The artifact training_roc_curve will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'
2022/09/28 19:02:04 WARNING mlflow.sklearn.utils: PrecisionRecallDisplay.from_estimator failed. The artifact training_precision_recall_curve will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'
[I 2022-09-28 19:02:06,875] Trial 0 finished with value: 0.6481270992053739 and parameters: {'clf__splitter': 'random', 'clf__max_depth': 105, 'clf__max_features': 'log2'}. Best is trial 0 with value: 0.6481270992053739.
202

In [18]:
print("Stats for the optimized Model")
print("Best Score:", study.best_value)
best_parameters = study.best_params
model_parameters = {}
for param in best_parameters:
    print(param, ":", best_parameters[param])
    model_parameters[param[5:]] = best_parameters[param]

tuned_model = DecisionTreeClassifier(**model_parameters)
tuned_model.fit(X_train, y_train)

pred = tuned_model.predict(X_test)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, pred, labels=labels)
print("labels:", labels)
print("Precision:", [round(x, 2) for x in precision])
print("recall:", [round(x, 2) for x in recall])
print("f1:", [round(x, 2) for x in f1])

2022/09/28 19:22:05 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'be135ac7d4b2421e8e85737903691271', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2022/09/28 19:22:05 WARNING mlflow.sklearn.utils: _get_classifier_artifacts.<locals>.plot_confusion_matrix failed. The artifact training_confusion_matrix will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'
2022/09/28 19:22:05 WARNING mlflow.sklearn.utils: RocCurveDisplay.from_estimator failed. The artifact training_roc_curve will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'
2022/09/28 19:22:05 WARNING mlflow.sklearn.utils: PrecisionRecallDisplay.from_estimator failed. The artifact training_precision_recall_curve will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'


Stats for the optimized Model
Best Score: 0.5776693188334563
clf__splitter : random
clf__max_depth : 10
clf__max_features : log2
labels: [0, 1]
Precision: [0.99, 0.05]
recall: [0.83, 0.45]
f1: [0.9, 0.09]


Modelo 4: Non-Linear SVC

In [19]:
model = Pipeline([
    ('clf', SVC())
])

def objective(trial):
    with mlflow.start_run() as run:

        clf__kernel = trial.suggest_categorical('clf__kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
        clf__gamma = trial.suggest_categorical('clf__gamma', ['scale', 'auto'])
        clf__C = trial.suggest_int('clf__C', 1, 50)

        params = {
            'clf__kernel': clf__kernel,
            'clf__gamma': clf__gamma,
            'clf__C': clf__C
        }
        
        model.set_params(**params)

        cv_score = np.mean(cross_val_score(model, X_train, y_train, cv=8, n_jobs=-1))
        mlflow.log_metric("cross_val_score", cv_score)

        model.fit(X_train, y_train)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(model, "model", registered_model_name="SVCModel")
        else:
            mlflow.sklearn.log_model(model, "model")

        pred = model.predict(X_test)
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, pred, labels=labels)

        precision = precision[1]
        recall = recall[1]
        f1 = f1[1]

        mlflow.log_metric("test_precision", precision)
        mlflow.log_metric("test_recall", recall)
        mlflow.log_metric("test_f1", f1)

    return cv_score

study = optuna.create_study()
study.optimize(objective, timeout=(6*training_time_in_seconds))

[I 2022-09-28 19:22:07,210] A new study created in memory with name: no-name-c18ae9cf-ad9e-484f-a414-7ebc8995bf02
2022/09/28 19:23:09 WARNING mlflow.sklearn.utils: _get_classifier_artifacts.<locals>.plot_confusion_matrix failed. The artifact training_confusion_matrix will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'
2022/09/28 19:23:09 WARNING mlflow.sklearn.utils: RocCurveDisplay.from_estimator failed. The artifact training_roc_curve will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'
2022/09/28 19:23:09 WARNING mlflow.sklearn.utils: PrecisionRecallDisplay.from_estimator failed. The artifact training_precision_recall_curve will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'
[I 2022-09-28 19:23:12,493] Trial 0 finished with value: 0.7286162652576391 and parameters: {'clf__kernel': 'linear', 'clf__gamma': 'scale', 'clf__C': 12}. Best is trial 0 with value: 0.7286162652576391.
2022/09/28 19:23:14 

In [20]:
print("Stats for the optimized Model")
print("Best Score:", study.best_value)
best_parameters = study.best_params
model_parameters = {}
for param in best_parameters:
    print(param, ":", best_parameters[param])
    model_parameters[param[5:]] = best_parameters[param]

tuned_model = SVC(**model_parameters)
tuned_model.fit(X_train, y_train)

pred = tuned_model.predict(X_test)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, pred, labels=labels)
print("labels:", labels)
print("Precision:", [round(x, 2) for x in precision])
print("recall:", [round(x, 2) for x in recall])
print("f1:", [round(x, 2) for x in f1])

2022/09/28 20:22:13 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '7ab696d8d4234843950ba8c9b8d04b29', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Stats for the optimized Model
Best Score: 0.3442722413369378
clf__kernel : sigmoid
clf__gamma : auto
clf__C : 3


2022/09/28 20:22:14 WARNING mlflow.sklearn.utils: _get_classifier_artifacts.<locals>.plot_confusion_matrix failed. The artifact training_confusion_matrix will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'
2022/09/28 20:22:14 WARNING mlflow.sklearn.utils: RocCurveDisplay.from_estimator failed. The artifact training_roc_curve will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'
2022/09/28 20:22:14 WARNING mlflow.sklearn.utils: PrecisionRecallDisplay.from_estimator failed. The artifact training_precision_recall_curve will not be recorded. Artifact error: 'NoneType' object has no attribute 'canvas'


labels: [0, 1]
Precision: [0.97, 0.01]
recall: [0.37, 0.33]
f1: [0.53, 0.02]


# Conclusões

Os modelos não alcançaram uma acurácia que consideramos útil. Acreditamos que os seguintes fatos contribuíram nisso:
- Dataset desbalanceado. Considerando que existiam muito mais labels negativas que positivas, a nossa primeira iteração de modelos sofreu muito com esse desbalanceamento. Basicamente, se o modelo aprendesse a "chutar" sempre numa label negativa, sem muita lógica por trás, ele acertava quase sempre. Fomos capazes de balancear o dataset, porém, cortando labels negativas para que tivéssemos a mesma quantidade de exemplos em cada classe. Porém, isso trouxe a sua própria gama de problemas.
- Tamanho do dataset de treino. Como informando no ponto anterior, balanceamos o dataset a custo de reduzí-lo. Chegamos a conclusão porém, que o número de exemplos foi baixíssimo e insuficiente para criar um modelo interessante. Acabou que, para treino, pudemos contar apenas com cerca de 3000 exemplos.

O que aprendemos?
Até agora, academicamente falando, todos os nossos encontros com a construção de modelos foram com datasets pré-determinados em condições que poderiam ser consideradas ótimas. Foi muito iluminante ter a chance de pegar um dataset "cru", ou "selvagem", mais perto do que podemos encontrar na vida real. Nós acabamos tendo uma ideia de um bom modelo sem validar se os dados disponíveis nos permitiriam construí-lo. Com certeza, daqui pra frente, a primeira coisa que checaremos na hora de trabalhar em cima do tópico, será o balanceamento das labels.

Em termos técnicos, também aprendemos a utilizar o optuna como uma alternativa mais inteligente de otimização de modelos, e o mlflow para registro e gerenciamento deles. Além, claro, das técnicas de limpeza e tratamento de dados que já vínhamos utilizando desde o primeiro projeto.
